In [38]:
import random
import numpy as np

Декодирование и исправление ошибок (7,4-код)

In [ ]:
def decode_with_error_correction(received, generator, max_errors):
    length = len(received)
    syndrome = np.polydiv(received, generator)[1] % 2

    for idx in range(length):
        error_pattern = np.zeros(length, dtype=int)
        error_pattern[length - idx - 1] = 1

        shifted_syndrome = np.polymul(syndrome, error_pattern) % 2
        residual = np.polydiv(shifted_syndrome, generator)[1] % 2

        if sum(residual) <= max_errors:
            correction = np.polymul(error_pattern, residual) % 2
            corrected_word = np.polyadd(received, correction) % 2
            decoded = np.polydiv(corrected_word, generator)[0] % 2
            return decoded.astype(int)

    return None


Кодирование сообщений

In [40]:
def encode(data, generator):
    return np.polymul(data, generator) % 2

Внесение случайных ошибок

In [41]:
def add_random_errors(word, error_count):
    length = len(word)
    error_positions = random.sample(range(length), error_count)
    print(f"Error positions: {error_positions}")
    for pos in error_positions:
        word[pos] ^= 1
    return word

In [42]:
def investigate_code_7_4():
    print("\nИсследование кода (7,4)\n")
    g = np.array([1, 1, 0, 1]) 
    t = 1  

    for error_count in range(1, 4): 
        word = np.array([1, 0, 1, 0])
        print(f"Исходное сообщение - {word}")
        codeword = encode(word, g)
        print(f"Закодированное сообщение - {codeword}")
        codeword_with_mistakes = add_random_errors(codeword.copy(), error_count)
        print(f"Сообщение с ошибками - {codeword_with_mistakes}")
        decoded = decode_with_error_correction(codeword_with_mistakes, g, t)
        print(f"Декодированное сообщение - {decoded}")
        if np.array_equal(word, decoded):
            print("Исходное и декодированное сообщения совпадают\n")
        else:
            print("Исходное и декодированное сообщения не совпадают.\n")

In [45]:
investigate_code_7_4()


Исследование кода (7,4)

Исходное сообщение - [1 0 1 0]
Закодированное сообщение - [1 1 1 0 0 1 0]
Error positions: [5]
Сообщение с ошибками - [1 1 1 0 0 0 0]
Декодированное сообщение - [1 0 1 0]
Исходное и декодированное сообщения совпадают

Исходное сообщение - [1 0 1 0]
Закодированное сообщение - [1 1 1 0 0 1 0]
Error positions: [0, 4]
Сообщение с ошибками - [0 1 1 0 1 1 0]
Декодированное сообщение - [0 1 0 0]
Исходное и декодированное сообщения не совпадают.

Исходное сообщение - [1 0 1 0]
Закодированное сообщение - [1 1 1 0 0 1 0]
Error positions: [3, 1, 4]
Сообщение с ошибками - [1 0 1 1 1 1 0]
Декодированное сообщение - [1 1 0 0]
Исходное и декодированное сообщения не совпадают.



In [46]:
def encode_message_15_9(u, g):
    return np.polymul(u, g) % 2

In [47]:
def add_errors(word, burst_length):
    length = len(word)
    start = random.randint(0, length - burst_length)
    for i in range(burst_length):
        word[(start + i) % length] ^= 1 
    print(f"Errors applied from {start} to {(start + burst_length - 1) % length}")
    return word


In [48]:
def is_correctable_error(error_pattern, max_length):
    trimmed = np.trim_zeros(error_pattern, 'f')
    trimmed = np.trim_zeros(trimmed, 'b')
    return len(trimmed) <= max_length and len(trimmed) > 0


In [49]:
def decode_15_9(received, generator, max_length):
    length = len(received)
    syndrome = np.polydiv(received, generator)[1] % 2 

    for idx in range(length):
        error_pattern = np.zeros(length, dtype=int)
        error_pattern[length - idx - 1] = 1

        shifted_syndrome = np.polymul(syndrome, error_pattern) % 2
        residual = np.polydiv(shifted_syndrome, generator)[1] % 2

        if is_correctable_error(residual, max_length):
            correction = np.polymul(error_pattern, residual) % 2
            corrected_word = np.polyadd(received, correction) % 2
            decoded = np.polydiv(corrected_word, generator)[0] % 2
            return decoded.astype(int)

    return None


In [50]:
def investigate_code_15_9():
    print("\nИсследование кода (15,9)\n")
    g = np.array([1, 0, 0, 1, 1, 1, 1])
    t = 3  # Пакеты ошибок длиной 3

    for packet_length in range(1, 5):
        word = np.array([1, 1, 0, 0, 0, 1, 0, 0, 0])
        print(f"Исходное сообщение: {word}")
        codeword = encode_message_15_9(word, g)
        print(f"Закодированное сообщение: {codeword}")
        codeword_with_pack_mistakes = add_errors(codeword.copy(), packet_length)
        print(f"Сообщение с пакетом ошибок: {codeword_with_pack_mistakes}")
        decoded = decode_15_9(codeword_with_pack_mistakes, g, t)
        print(f"Декодированное сообщение: {decoded}")
        if np.array_equal(word, decoded):
            print("Исходное сообщение и декодированное совпадают.\n")
        else:
            print("Исходное сообщение и декодированное не совпадают.\n")

In [52]:
investigate_code_15_9()


Исследование кода (15,9)

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Errors applied from 0 to 0
Сообщение с пакетом ошибок: [0 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Декодированное сообщение: [0 1 0 1 1 0 0 0 1]
Исходное сообщение и декодированное не совпадают.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Errors applied from 13 to 14
Сообщение с пакетом ошибок: [1 1 0 1 0 1 0 1 1 1 1 1 0 1 1]
Декодированное сообщение: [1 1 0 0 0 1 0 0 0]
Исходное сообщение и декодированное совпадают.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Errors applied from 9 to 11
Сообщение с пакетом ошибок: [1 1 0 1 0 1 0 1 1 0 0 0 0 0 0]
Декодированное сообщение: [1 1 0 0 0 1 0 0 0]
Исходное сообщение и декодированное совпадают.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Errors applied from 1 to 4
Сообщ